<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Operations on <span style="font-weight:bold; color:green">Spark Dataframe</span> using Python</b></div><hr>
<div style="text-align:right;">Sergei Yu. Papulin <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Content</span>
    <ol>
        <li><a href="#1">Creating Dataframe</a></li>
        <li><a href="#2">Access To Info About Dataframe</a></li>
        <li><a href="#3">Access To Dataframe</a></li>
        <li><a href="#4">Manipulating Columns</a></li>
        <li><a href="#5">Filtering And Manipulating Data</a></li>
        <li><a href="#6">Manipulating Dataframes</a></li>
        <li><a href="#7">Manipulating Dataframe Partitions</a></li>
        <li><a href="#8">Caching Data In Memory</a></li>
        <li><a href="#9">SQL Expression</a></li>
        <li><a href="#10">Load/Save Functions</a></li>
        <li><a href="#11">References</a></li>
    </ol>
</div>

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Creating Dataframe</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

<p><b>2D Array and RDD</b></p>

In [ ]:
import os
import sys

os.environ["SPARK_HOME"]="/usr/lib/spark"
os.environ["PYSPARK_PYTHON"]="/opt/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/opt/anaconda3/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
conf = pyspark.SparkConf() \
        .set("spark.executor.memory", "1g") \
        .set("spark.executor.core", "2") \
        .setMaster("local[2]")

In [ ]:
spark = SparkSession \
    .builder \
    .config(conf=conf) \
    .getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
from pyspark.sql import Row

In [ ]:
data = [[0, "Dima", "Moscow", 1988, 4, "m"],
           [1, "Sveta", "Kiev", 1999, 4, "f"],
           [2, "Alex", "Minsk", 1954, None, "m"],
           [3, "Ivan", "St.Petersburg", 2005, 6, "m"],
           [4, "Kate", "London", 2001, None, "f"],
           [5, "Maria", "New York", 1997, 7, "f"]]
data

In [ ]:
rdd_data = sc.parallelize(data, 2)
rdd_data.take(5)

In [ ]:
rdd_data_indx = rdd_data.zipWithIndex()
rdd_data_indx.take(5)

In [ ]:
Person = Row("Id", "Name", "City", "Year", "Grade", "Gender")

In [ ]:
rdd_data_row = rdd_data.map(lambda x: Person(x[0], x[1], x[2], x[3], x[4], x[5]))
rdd_data_row.take(5)

In [ ]:
# Case 1

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [ ]:
schema = StructType([StructField(name="Id", dataType=IntegerType(), nullable=False),
                     StructField("Name", StringType(), True),
                     StructField("City", StringType(), True),
                     StructField("Year", IntegerType(), True),
                     StructField("Grade", IntegerType(), True),
                     StructField("Gender", StringType(), True)])

In [ ]:
df_data = spark.createDataFrame(rdd_data_row, schema)
df_data.show(5)

In [ ]:
df_data.printSchema()

In [ ]:
# Case 2

In [ ]:
df_data_toDF = rdd_data_row.toDF()
df_data_toDF.show(5)

In [ ]:
df_data_toDF.printSchema()

In [ ]:
# df_data.explain(True)

<p><b>Read data from a file</b></p>

In [ ]:
# S3 FS
file_path = "s3://YOUR_BUCKET/data/spark_dataframe/persons.csv"

# Local FS
file_path = "file:///YOUR_LOCAL_PATH/data/persons.csv"

<p>Through an intermediate RDD and schema</p>

In [ ]:
rdd_data = sc.textFile(file_path, 2)
rdd_data.take(6)

In [ ]:
def parse_row(row):
    x = row.split(",")
    return Person(int(x[0]), x[1], x[2], int(x[3]), None if x[4] in {"null", ""} else int(x[4]), x[5])

rdd_data_row = rdd_data.map(parse_row)
rdd_data_row.take(6)

In [ ]:
df_data = spark.createDataFrame(rdd_data_row, schema)
df_data.show(6)

<p>Specifying the data source</p>

In [ ]:
df = spark.read.load(path=file_path, 
                          format="csv",
                          schema=schema,
                          header="false", 
                          inferSchema="false", sep=",", nullValue="null", mode="DROPMALFORMED")
df.show()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Access To Info About Dataframe</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

<p>Show a dataframe schema</p>

In [ ]:
df_data.printSchema()

In [ ]:
df_data.schema

<p>Columns</p>

In [ ]:
df_data.columns

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Access To Dataframe</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

In [ ]:
df_data.show(4)

In [ ]:
df_data.first()

In [ ]:
df_data.take(4)

In [ ]:
df_data.collect()

In [ ]:
data_rdd = df_data.rdd
data_rdd.take(4)

<p>The number of records in the dataframe</p>

In [ ]:
df_data.count()

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Manipulating Columns</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

<p>Select columns</p>

In [ ]:
df_data[["Name", "City"]].show()

In [ ]:
df_data.select(["Name", "City"]).show()

<p>Insert a column</p>

<p><i>Approach 1</i></p>

In [ ]:
df_data_new_clmn = df_data.withColumn("New Column", df_data["Grade"]+1)
df_data_new_clmn.show()

<p><i>Approach 2</i></p>

In [ ]:
df_data.select("*", (df_data["Grade"]+1).alias("New Column 1"), (df_data["Grade"]+2).alias("New Column 2")).show()

<p>Rename a column</p>

In [ ]:
df_data.withColumnRenamed("Name", "Name New").show()

<p>Delete a column</p>

In [ ]:
df_data.drop("Grade").show()

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">5. Filtering And Manipulating Data</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

<p>Filter records</p>

In [ ]:
df_data.filter(df_data["Name"]=="Alex").show()

In [ ]:
import pyspark.sql.functions as F

In [ ]:
df_data.filter(F.col("Name")=="Alex").show()

<p>Use filter expressions</p>

In [ ]:
filter_expr_or = (F.col("Gender")=="m") | (F.col("Grade")>4)
filter_expr_and = (F.col("Gender")=="m") & (F.col("Grade")>=4)

In [ ]:
df_data.filter(filter_expr_or).show()

In [ ]:
df_data.filter(filter_expr_and).show()

In [ ]:
df_data.select(F.col("Id"), F.col("Name")).where(filter_expr_and).show()

<p>Use the LIKE condition</p>

In [ ]:
df_data.filter(F.lower(F.col("Name")).like("%le%")).show()

<p>Find distinct values by columns</p>

In [ ]:
df_data[["Gender"]].distinct().show()

In [ ]:
df_data[["Grade", "Gender"]].distinct().show()

<p>Determine the number of unique values by columns</p>

In [ ]:
df_data.select(F.countDistinct("Gender"), F.countDistinct("Name")).show()

<p>Aggregate by a column</p>

In [ ]:
df_data.agg(F.countDistinct("Gender")).show()

In [ ]:
df_data.groupBy("Gender").agg(F.min("Grade"), F.max("Grade"), F.avg("Grade")).show()

In [ ]:
df_data.groupBy("Gender").agg({"Grade": "min"}).show()

<p>Sort</p>

In [ ]:
df_data.sort("Name", ascending=True).show()

In [ ]:
df_data.sort(F.desc("Name")).show()

In [ ]:
df_data.orderBy(F.asc("Name")).groupBy("Gender").agg(F.collect_list("Name")).show()

<p>Replace values in columns</p>

In [ ]:
df_data.na.replace(["Kate", "Alex"], ["Kate New", "Alex New"], "Name").show()

<p>Remove NA values</p>

In [ ]:
df_data.na.drop(how="any", thresh=None, subset=None).show()

In [ ]:
df_data.na.drop(how="all", thresh=None, subset=None).show()

In [ ]:
df_data.na.drop(how="all", thresh=None, subset="Grade").show()

In [ ]:
df_data.na.drop(how="all", thresh=1, subset="Grade").show()

<p>Replace NA values</p>

In [ ]:
df_data.na.fill(-1, subset="Grade").show()

In [ ]:
df_data.na.fill({"Grade": -1}).show()

<a name="6"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">6. Manipulating Dataframes</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

<p>Union</p>

In [ ]:
# S3 FS
file_person_add = "s3://YOUR_BUCKET/data/spark_dataframe/persons_addition.csv"

# Local FS
file_person_add = "file:///YOUR_LOCAL_PATH/data/persons_addition.csv"

In [ ]:
df_data_add = spark.read.load(path=file_person_add, 
                              format="csv",
                              schema=schema,
                              header="false", 
                              inferSchema="false", sep=",", nullValue="null", mode="DROPMALFORMED")
df_data_add.show()

In [ ]:
df_data.show()

In [ ]:
df_data.union(df_data_add).show()

In [ ]:
df_data.union(df_data_add).distinct().show()

<p>Intersect</p>

In [ ]:
df_data.intersect(df_data_add).show()

<p>Subtract</p>

In [ ]:
df_data.subtract(df_data_add).show()

In [ ]:
df_data_add.subtract(df_data).show()

<p>Join</p>

In [ ]:
# S3 FS
file_items_path = "s3://YOUR_BUCKET/data/spark_dataframe/person_items.csv"

# Local FS
file_items_path = "file:///YOUR_LOCAL_PATH/data/person_items.csv"

In [ ]:
schema_item = StructType([StructField(name="Id", dataType=IntegerType(), nullable=False),
                          StructField("Item", StringType(), True)])

In [ ]:
df_items = spark.read.load(path=file_items_path, 
                           format="csv",
                           schema=schema_item,
                           header="false", 
                           inferSchema="false", sep=",", nullValue="null", mode="DROPMALFORMED")
df_items.show()

In [ ]:
df_data.join(df_items, on="Id", how="inner").show() #how = [inner, outer, left_outer, right_outer, leftsemi]

<a name="7"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">7. Manipulating Dataframe Partitions</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

<p>Change the number of partitions</p>

In [ ]:
df_data.rdd.getNumPartitions()

In [ ]:
df_data_coal = df_data.coalesce(1)

In [ ]:
df_data_coal.rdd.getNumPartitions()

In [ ]:
df_data_repart = df_data.repartition(3)

In [ ]:
df_data_repart.rdd.getNumPartitions()

<p>Other operations</p>

In [ ]:
foreachPartition
sortWithinPartitions

<a name="8"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">8. Caching Data In Memory
</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

In [ ]:
df_data.persist()

In [ ]:
df_data.unpersist()

<a name="9"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">9. SQL Expression
</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

In [ ]:
df_data.createOrReplaceTempView("persons")
df_grade = spark.sql("SELECT * FROM persons WHERE grade > 4")
df_grade.show()

<a name="10"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">10. Load/Save Functions
</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

In [ ]:
# S3 FS
output_path = "s3://YOUR_BUCKET/data/spark_dataframe/output"

# Local FS
output_path = "file:///YOUR_LOCAL_PATH/data/output"

In [ ]:
df_data.coalesce(1).write.format("csv").save(path=output_path)

In [ ]:
df_data_saved = spark.read.load(path=output_path+"/par*", 
                                format="csv",
                                schema=schema,
                                header="false", 
                                inferSchema="false", sep=",", nullValue="null", mode="DROPMALFORMED")
df_data_saved.show()

<a name="11"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">11. References
</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To Content</a></div>
    </div>
</div>

<a href="https://spark.apache.org/docs/latest/sql-programming-guide.html">Spark SQL, DataFrames and Datasets Guide</a><br>
<a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html">pyspark.sql module</a>